# Chapter 2: Our First Model

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from PIL import Image, ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES=True

## Setting up DataLoaders

We'll use the built-in dataset of `torchvision.datasets.ImageFolder` to quickly set up some dataloaders of downloaded cat and fish images. 

`check_image`  is a quick little function that is passed to the `is_valid_file` parameter in the ImageFolder and will do a sanity check to make sure PIL can actually open the file. We're going to use this in lieu of cleaning up the downloaded dataset.


In [16]:
def check_image(path):
    try:
        im = Image.open(path)
        return True
    except:
        return False
check_image("../images/test/cat/3156111_a9dba42579.jpg")

True

Set up the transforms for every image:

* Resize to 64x64
* Convert to tensor
* Normalize using ImageNet mean & std


In [17]:
img_transforms = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225] )
    ])

# Question : How to calculate the mean and std ?

In [42]:
train_data_path = "./images/train/"
train_data = torchvision.datasets.ImageFolder(root=train_data_path,transform=img_transforms )#, is_valid_file=check_image)

In [43]:
val_data_path = "./images/val/"
val_data = torchvision.datasets.ImageFolder(root=val_data_path,transform=img_transforms) #is_valid_file=check_image)

In [44]:
test_data_path = "./images/test/"
test_data = torchvision.datasets.ImageFolder(root=test_data_path,transform=img_transforms)#is_valid_file=check_image)

In [45]:
batch_size=64

In [46]:
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)
val_data_loader  = torch.utils.data.DataLoader(val_data, batch_size=batch_size) 
test_data_loader  = torch.utils.data.DataLoader(test_data, batch_size=batch_size) 

## Our First Model, SimpleNet

SimpleNet is a very simple combination of three Linear layers and ReLu activations between them. Note that as we don't do a `softmax()` in our `forward()`, we will need to make sure we do it in our training function during the validation phase.

In [47]:
class SimpleNet(nn.Module):

    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(12288, 84) # 64*64*3
        self.fc2 = nn.Linear(84, 50)
        self.fc3 = nn.Linear(50,2)
    
    def forward(self, x):
        x = x.view(-1, 12288)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [48]:
simplenet = SimpleNet()

## Create an optimizer

Here, we're just using Adam as our optimizer with a learning rate of 0.001.

In [49]:
optimizer = optim.Adam(simplenet.parameters(), lr=0.001)

## Copy the model to GPU

Copy the model to the GPU if available.

In [50]:
if torch.cuda.is_available():
    device = torch.device("cuda") 
else:
    device = torch.device("cpu")

simplenet.to(device)

SimpleNet(
  (fc1): Linear(in_features=12288, out_features=84, bias=True)
  (fc2): Linear(in_features=84, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=2, bias=True)
)

## Training 

Trains the model, copying batches to the GPU if required, calculating losses, optimizing the network and perform validation for each epoch.

In [51]:
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device="cpu"):
    for epoch in range(1, epochs+1):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * inputs.size(0)
        training_loss /= len(train_loader.dataset)
        
        model.eval()
        num_correct = 0 
        num_examples = 0
        for batch in val_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            output = model(inputs)
            targets = targets.to(device)
            loss = loss_fn(output,targets) 
            valid_loss += loss.data.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1], targets)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(val_loader.dataset)

        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}'.format(epoch, training_loss,
        valid_loss, num_correct / num_examples))

In [58]:
train(simplenet, optimizer,torch.nn.CrossEntropyLoss(), train_data_loader,val_data_loader, epochs=20, device=device)

Epoch: 1, Training Loss: 0.37, Validation Loss: 1.31, accuracy = 0.54
Epoch: 2, Training Loss: 0.37, Validation Loss: 1.31, accuracy = 0.54
Epoch: 3, Training Loss: 0.37, Validation Loss: 1.31, accuracy = 0.54
Epoch: 4, Training Loss: 0.37, Validation Loss: 1.31, accuracy = 0.54
Epoch: 5, Training Loss: 0.37, Validation Loss: 1.31, accuracy = 0.54
Epoch: 6, Training Loss: 0.37, Validation Loss: 1.31, accuracy = 0.54
Epoch: 7, Training Loss: 0.37, Validation Loss: 1.31, accuracy = 0.54
Epoch: 8, Training Loss: 0.37, Validation Loss: 1.31, accuracy = 0.54
Epoch: 9, Training Loss: 0.37, Validation Loss: 1.31, accuracy = 0.54
Epoch: 10, Training Loss: 0.37, Validation Loss: 1.31, accuracy = 0.54
Epoch: 11, Training Loss: 0.37, Validation Loss: 1.31, accuracy = 0.54
Epoch: 12, Training Loss: 0.37, Validation Loss: 1.31, accuracy = 0.54
Epoch: 13, Training Loss: 0.37, Validation Loss: 1.31, accuracy = 0.54
Epoch: 14, Training Loss: 0.37, Validation Loss: 1.31, accuracy = 0.54
Epoch: 15, Trai

## Making predictions

Labels are in alphanumeric order, so `cat` will be 0, `fish` will be 1. We'll need to transform the image and also make sure that the resulting tensor is copied to the appropriate device before applying our model to it.

In [63]:
labels = ['cat','fish']

img = Image.open("./images/val/fish/100_1422.JPG")
img = img_transforms(img).to(device)
print(img.shape)
img = torch.unsqueeze(img, 0)
print(img.shape)
"""
However, because our network uses batches, it actually expects a 4D tensor,
with the first dimension denoting the different images within a batch.
We don’t have a batch, but we can create a batch of length 1 by using unsqueeze(0),
which adds a new dimension at the front of our tensor.
"""


simplenet.eval()
print(simplenet(img))
prediction = F.softmax(simplenet(img), dim=1)
print(prediction)
prediction = prediction.argmax()
print(prediction)# the bigger probability value
print(labels[prediction])
print(labels)

torch.Size([3, 64, 64])
torch.Size([1, 3, 64, 64])
tensor([[-5.0232,  6.7720]], grad_fn=<AddmmBackward0>)
tensor([[7.5407e-06, 9.9999e-01]], grad_fn=<SoftmaxBackward0>)
tensor(1)
fish
['cat', 'fish']


## Saving Models

We can either save the entire model using `save` or just the parameters using `state_dict`. Using the latter is normally preferable, as it allows you to reuse parameters even if the model's structure changes (or apply parameters from one model to another).

In [56]:
torch.save(simplenet, "simplenet")
simplenet = torch.load("simplenet")


In [57]:
torch.save(simplenet.state_dict(), "simplenet")
simplenet = SimpleNet()
simplenet_state_dict = torch.load("simplenet")
simplenet.load_state_dict(simplenet_state_dict)   


<All keys matched successfully>